In [ ]:
#getting all the packages
import os
import sys

print(f'os.environ={os.environ}')
print(f'sys.argv={sys.argv}')
print(f"MLFLOW_TRACKING_URI{os.environ.get('MLFLOW_TRACKING_URI')}")
print(f"sys.executable={sys.executable}")
print(f"sys.path={sys.path}")

os.environ=environ({'NV_LIBCUBLAS_DEV_VERSION': '11.4.1.1043-1', 'NV_CUDA_COMPAT_PACKAGE': 'cuda-compat-11-2', 'NV_CUDNN_PACKAGE_DEV': 'libcudnn8-dev=8.1.1.33-1+cuda11.2', 'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64', 'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.8.4-1+cuda11.2', 'TCLLIBPATH': '/usr/share/tcltk/tcllib1.19', 'CLOUDSDK_PYTHON': 'python3', 'LANG': 'en_US.UTF-8', 'NV_LIBNPP_DEV_PACKAGE': 'libnpp-dev-11-2=11.3.2.152-1', 'HOSTNAME': '613edc38c773', 'OLDPWD': '/', 'CLOUDSDK_CONFIG': '/content/.config', 'NV_LIBNPP_VERSION': '11.3.2.152-1', 'NV_NVPROF_DEV_PACKAGE': 'cuda-nvprof-11-2=11.2.152-1', 'NVIDIA_VISIBLE_DEVICES': 'all', 'NV_NVPROF_VERSION': '11.2.152-1', 'NV_LIBCUSPARSE_VERSION': '11.4.1.1152-1', 'DATALAB_SETTINGS_OVERRIDES': '{"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=172.28.0.2"],"debugAdapterMultiplexerPath":"/usr/local/bin/dap_multiplexer","enableLsp":true}', 'NV_LIBCUBLAS_DEV_PACKAGE': 'libcublas-dev-1

This notebook will cover the baseline model buuilding for the Thunderbird dataset, we will cover the following algorithm for the baseline model 
1.)PCA- principal component analysis, 
2.)One-Class SVM- commonly used one-class classification model and commonplace in log anomaly detection 
3.)Isolation Forest -  this algorithm uses an unsupervised methodology for anomaly detection using features as tree structure


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


We will be using the logbert github for dataparsing and baseline analysis, https://github.com/HelenGuohx/logbert.git

In [ ]:
import os
! git clone https://github.com/HelenGuohx/logbert.git


Cloning into 'logbert'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 130 (delta 9), reused 5 (delta 5), pack-reused 90
Receiving objects: 100% (130/130), 210.44 KiB | 6.58 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
import sys
sys.path.append('/content/logbert/')
from loglizer.models import InvariantsMiner, PCA, IsolationForest, OneClassSVM, LogClustering, LR, SVM
from loglizer import dataloader, preprocessing
from loglizer.utils import metrics
from logdeep.dataset.session import sliding_window

In [ ]:
#%cd logbert

import os
import gc
import pandas as pd
import numpy as np

import argparse
from tqdm import tqdm
import argparse
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.utils import shuffle



In [ ]:
#train test split 
ouput_dir = "/content/gdrive/MyDrive/W266_Final_Project/Tbird parsed/"
middle_dir = ""
log_file = "Thunderbird_20M.log"

(x_train, y_train), (x_test, y_test) = dataloader.load_data(ouput_dir, middle_dir, log_file, is_mapping=True)
feature_extractor = preprocessing.FeatureExtractor()
x_train = feature_extractor.fit_transform(x_train)
x_test = feature_extractor.transform(x_test)

/content/logbert/loglizer/dataloader.py:286: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train = np.array(train).reshape(-1,1)
/content/logbert/loglizer/dataloader.py:292: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_normal = np.array(test_normal).reshape(-1,1)
/content/logbert/loglizer/dataloader.py:298: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' wh

Train normal size: 6000
Train abnormal size: 3000
Total logkey(exclude 0:UNK) 1206
Test normal size: 71155
Test abnormal size: 42385
num_unk_event in test data: 0
====== Transformed train data summary ======
Train data shape: 9000-by-921

====== Transformed test data summary ======
Test data shape: 113540-by-921



In [ ]:
x_train[3]

array([3., 5., 3., 8., 4., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.,
       1., 5., 4., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
#PCA analysis
%%time
print("="*20 + " Model: PCA " + "="*20)
for th in np.arange(1):
    print("theshold", th)
    model = PCA(n_components=0.8, threshold=1, c_alpha = 1.9600)
    model.fit(x_train)
    print('Train validation:')
    precision, recall, f1 = model.evaluate(x_train, y_train)
    print('Test validation:')
    precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: PCA ====================
theshold 0
====== Model summary ======
n_components: 5
Project matrix shape: 921-by-921
SPE threshold: 1

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 3000, FP: 5996, TN: 4, FN: 0
Precision: 33.348%, recall: 100.000%, F1-measure: 50.017%

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 42385, FP: 71091, TN: 64, FN: 0
Precision: 37.352%, recall: 100.000%, F1-measure: 54.388%

CPU times: user 34.4 s, sys: 2.75 s, total: 37.1 s
Wall time: 19.1 s


In [ ]:
#isolation forest
%%time
print("="*20 + " Model: IsolationForest " + "="*20)
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=19)
model.fit(x_train)
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: IsolationForest ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 49, FP: 104, TN: 5896, FN: 2951
Precision: 32.026, recall: 1.633, F1-measure: 3.108

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 686, FP: 1297, TN: 69858, FN: 41699
Precision: 34.594, recall: 1.619, F1-measure: 3.092

CPU times: user 35.7 s, sys: 8.31 s, total: 44 s
Wall time: 44 s


In [ ]:
#SVM
%%time
print("="*20 + " Model: one class SVM " + "="*20)
model = OneClassSVM(kernel='rbf')
model.fit(x_train, y_train)

print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: one class SVM ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 976, FP: 6000, TN: 0, FN: 2024
Precision: 13.991, recall: 32.533, F1-measure: 19.567

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 14728, FP: 71155, TN: 0, FN: 27657
Precision: 17.149, recall: 34.748, F1-measure: 22.964

CPU times: user 8min 38s, sys: 631 ms, total: 8min 39s
Wall time: 8min 37s


In [ ]:
#Clustering
%%time
print("="*20 + " Model: LogClustering " + "="*20)
max_dist = 0.3  # the threshold to stop the clustering process
anomaly_threshold = 0.3  # the threshold for anomaly detection
model = LogClustering(max_dist=max_dist, anomaly_threshold=anomaly_threshold)
model.fit(x_train[y_train == 0, :])  # Use only normal samples for training
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: LogClustering ====================
====== Model summary ======
Starting offline clustering...
Processed 1000 instances.
Found 75 clusters offline.

Starting online clustering...
Processed 2000 instances.
Processed 4000 instances.
Processed 6000 instances.
Processed 6000 instances.
Found 110 clusters online.

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 1348, FP: 1, TN: 5999, FN: 1652
Precision: 99.926, recall: 44.933, F1-measure: 61.991

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 19242, FP: 376, TN: 70779, FN: 23143
Precision: 98.083, recall: 45.398, F1-measure: 62.068

CPU times: user 2min 53s, sys: 713 ms, total: 2min 54s
Wall time: 2min 54s
